In [1]:
from qaravan.tensorQ import *
from qaravan.core import *

In [2]:
# testing if we can prepare GHZ state
gate_list = [H(0), CNOT([1,0], 3), CNOT([2,1], 3)]
circ = Circuit(gate_list, 3)
print(circ)

sv = StatevectorSim(circ)
final_state = sv.run()
print(sv)

H gate on site(s) [0] 
bottom heavy CNOT2 gate on site(s) [0, 1] 
bottom heavy CNOT2 gate on site(s) [1, 2]


100%|██████████| 3/3 [00:00<00:00, 1009.14it/s]

0.7071|000⟩ + 0.7071|111⟩


In [3]:
# testing local expectations
local_ops = [pauli_Z, pauli_Z, pauli_Z]
sv.local_expectation(local_ops) # should be zero

array(0.)

In [4]:
# testing local expectations again
local_ops = [pauli_X, pauli_X, pauli_X]
sv.local_expectation(local_ops) # should be 1

array(1.)

In [5]:
# testing environments 
n = 4
sv = random_sv(n)

gate_list = [H(0), CNOT([1,0], n), CNOT([2,1], n), CNOT([3,2], n)]
circ = Circuit(gate_list, n)
sim = StatevectorSim(circ)
final_state = sim.run(progress_bar=False).reshape(2**n)
overlap_a = sv.conj().T @ final_state

for gate_idx in range(len(circ)):
    env, mat = sv_environment(circ, sv, gate_idx)
    overlap_b = np.trace(env @ mat)
    print(np.allclose(overlap_a, overlap_b))

True
True
True
True


In [3]:
# testing environment based state prep
skeleton = [[0, 1], [2, 3], [1, 2]]
target_sv = random_sv(4)

context = RunContext(
    progress_interval=15,
    max_iter=50,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle"
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    step, circ, cost_list = pickle.load(f)
step, cost_list

Step 0 at time 2025-04-15T17:01:37: cost = 0.06073033378479509
[Checkpoint saved at step 0]
Plateau with cost 0.01803293198372291 at step 5


(0, [np.float64(0.2350969136715479), np.float64(0.06073033378479509)])

In [ ]:
# resuming from previous run
context = RunContext(
    progress_interval=15,
    max_iter=50,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    resume=True
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    step, circ, cost_list = pickle.load(f)
step, cost_list

[Resuming from checkpoint: step 0]
Step 0 at time 2025-04-15T17:01:49: cost = 0.018303392244950922
[Checkpoint saved at step 0]
Plateau with cost 0.01803293198372291 at step 4


(0,
 [np.float64(0.2350969136715479),
  np.float64(0.06073033378479509),
  np.float64(0.018303392244950922)])